## Imports

In [253]:
import pandas as pd

from sklearn.neighbors import NearestNeighbors

## Data load

In [254]:
ROOT_PATH = '../'
DATASET_NAME = 'ebnerd_demo'

In [255]:
articles_parquet = pd.read_parquet(f'{ROOT_PATH}/data/{DATASET_NAME}/articles.parquet')
articles_parquet.head()

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,...,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3037230,Ishockey-spiller: Jeg troede jeg skulle dø,ISHOCKEY: Ishockey-spilleren Sebastian Harts h...,2023-06-29 06:20:57,False,Ambitionerne om at komme til USA og spille ish...,2003-08-28 08:55:00,None,article_default,https://ekstrabladet.dk/sport/anden_sport/isho...,...,[],"[Kriminalitet, Kendt, Sport, Katastrofe, Mindr...",142,"[327, 334]",sport,NaN,NaN,NaN,0.9752,Negative
1,3044020,Prins Harry tvunget til dna-test,Hoffet tvang Prins Harry til at tage dna-test ...,2023-06-29 06:21:16,False,Den britiske tabloidavis The Sun fortsætter me...,2005-06-29 08:47:00,"[3097307, 3097197, 3104927]",article_default,https://ekstrabladet.dk/underholdning/udlandke...,...,"[PER, PER]","[Kriminalitet, Kendt, Underholdning, Personfar...",414,[432],underholdning,NaN,NaN,NaN,0.7084,Negative
2,3057622,Rådden kørsel på blå plader,Kan ikke straffes: Udenlandske diplomater i Da...,2023-06-29 06:21:24,False,Slingrende spritkørsel. Grove overtrædelser af...,2005-10-10 07:20:00,[3047102],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Kriminalitet, Transportmiddel, Bil]",118,[133],nyheder,NaN,NaN,NaN,0.9236,Negative
3,3073151,Mærsk-arvinger i livsfare,FANGET I FLODBØLGEN: Skibsrederens oldebørn må...,2023-06-29 06:21:38,False,To oldebørn af skibsreder Mærsk McKinney Mølle...,2005-01-04 06:59:00,"[3067474, 3067478, 3153705]",article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Erhverv, Privat virksomhed, Livsstil, Familie...",118,[133],nyheder,NaN,NaN,NaN,0.9945,Negative
4,3193383,Skød svigersøn gennem babydyne,44-årig kvinde tiltalt for drab på ekssvigersø...,2023-06-29 06:22:57,False,En 44-årig mormor blev i dag fremstillet i et ...,2003-09-15 15:30:00,None,article_default,https://ekstrabladet.dk/krimi/article3193383.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9966,Negative


In [256]:
history_parquet = pd.read_parquet(f'{ROOT_PATH}/data/{DATASET_NAME}/train/history.parquet')
history_parquet.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,13538,"[2023-04-27T10:17:43.000000, 2023-04-27T10:18:...","[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11..."
1,58608,"[2023-04-27T18:48:09.000000, 2023-04-27T18:48:...","[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,..."
2,95507,"[2023-04-27T15:20:28.000000, 2023-04-27T15:20:...","[60.0, 100.0, 100.0, 21.0, 29.0, 67.0, 49.0, 5...","[9739035, 9738646, 9634967, 9738902, 9735495, ...","[18.0, 29.0, 51.0, 12.0, 10.0, 10.0, 13.0, 24...."
3,106588,"[2023-04-27T08:29:09.000000, 2023-04-27T08:29:...","[24.0, 57.0, 100.0, nan, nan, 100.0, 100.0, 73...","[9738292, 9738216, 9737266, 9737556, 9737657, ...","[9.0, 15.0, 42.0, 9.0, 3.0, 58.0, 26.0, 214.0,..."
4,617963,"[2023-04-27T14:42:25.000000, 2023-04-27T14:43:...","[100.0, 100.0, nan, 46.0, 23.0, 19.0, 61.0, 70...","[9739035, 9739088, 9738902, 9738968, 9738760, ...","[45.0, 29.0, 116.0, 26.0, 34.0, 42.0, 58.0, 59..."


In [257]:
behaviors_parquet = pd.read_parquet(f'{ROOT_PATH}/data/{DATASET_NAME}/train/behaviors.parquet', engine='pyarrow')

behaviors_parquet.head()

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0
1,152513,9778745.0,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, 9778623, 9089120, 9778661, ...",[9778661],150224,False,NaN,NaN,NaN,False,298,2.0,48.0
2,155390,NaN,2023-05-24 07:30:33,45.0,NaN,1,"[9778369, 9777856, 9778500, 9778021, 9778627, ...",[9777856],160892,False,NaN,NaN,NaN,False,401,215.0,100.0
3,214679,NaN,2023-05-23 05:25:40,33.0,NaN,2,"[9776715, 9776406, 9776566, 9776071, 9776808, ...",[9776566],1001055,False,NaN,NaN,NaN,False,1357,40.0,47.0
4,214681,NaN,2023-05-23 05:31:54,21.0,NaN,2,"[9775202, 9776855, 9776688, 9771995, 9776583, ...",[9776553],1001055,False,NaN,NaN,NaN,False,1358,5.0,49.0


## Build dataframe with users and viewed articles

In [258]:
user_viewed_articles = history_parquet.explode(['scroll_percentage_fixed', 'article_id_fixed', 'read_time_fixed']).reset_index(drop=True)
user_viewed_articles.drop(columns=['impression_time_fixed'], inplace=True)

user_viewed_articles.head()

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,13538,100.0,9738663,17.0
1,13538,35.0,9738569,12.0
2,13538,100.0,9738663,4.0
3,13538,24.0,9738490,5.0
4,13538,100.0,9738663,4.0


## Build dataframe with users and skipped articles

In [259]:
behaviors_parquet['article_ids_skipped'] = list(behaviors_parquet['article_ids_inview'].map(set) - behaviors_parquet['article_ids_clicked'].map(set))

behaviors_parquet.head()

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage,article_ids_skipped
0,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0,"{9775331, 9775402, 9771051, 9770028, 9775371, ..."
1,152513,9778745.0,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, 9778623, 9089120, 9778661, ...",[9778661],150224,False,NaN,NaN,NaN,False,298,2.0,48.0,"{9080070, 9420172, 9777492, 9717914, 9718298, ..."
2,155390,NaN,2023-05-24 07:30:33,45.0,NaN,1,"[9778369, 9777856, 9778500, 9778021, 9778627, ...",[9777856],160892,False,NaN,NaN,NaN,False,401,215.0,100.0,"{9778369, 9778627, 9778500, 9778021, 9778628, ..."
3,214679,NaN,2023-05-23 05:25:40,33.0,NaN,2,"[9776715, 9776406, 9776566, 9776071, 9776808, ...",[9776566],1001055,False,NaN,NaN,NaN,False,1357,40.0,47.0,"{9776071, 9776808, 9776855, 9776715, 9776046, ..."
4,214681,NaN,2023-05-23 05:31:54,21.0,NaN,2,"[9775202, 9776855, 9776688, 9771995, 9776583, ...",[9776553],1001055,False,NaN,NaN,NaN,False,1358,5.0,49.0,"{9776449, 9776583, 9776071, 9776855, 9771995, ..."


In [260]:
user_skipped_articles = behaviors_parquet.explode('article_ids_skipped').reset_index(drop=True)
user_skipped_articles.head()

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage,article_ids_skipped
0,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0,9775331
1,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0,9775402
2,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0,9771051
3,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0,9770028
4,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0,9775371


In [261]:
user_skipped_articles = user_skipped_articles[['user_id', 'article_ids_skipped']]
user_skipped_articles['read_time_fixed'] = 0
user_skipped_articles['scroll_percentage_fixed'] = 0

user_skipped_articles.rename(columns={'article_ids_skipped': 'article_id_fixed'}, inplace=True)

user_skipped_articles.head()

,user_id,article_id_fixed,read_time_fixed,scroll_percentage_fixed
0,22779,9775331,0,0
1,22779,9775402,0,0
2,22779,9771051,0,0
3,22779,9770028,0,0
4,22779,9775371,0,0


## Concat the two dataframes

In [262]:
user_articles_all = pd.concat([user_skipped_articles, user_viewed_articles])

In [263]:
len(user_articles_all)

511628

In [264]:
user_articles_all.head()

,user_id,article_id_fixed,read_time_fixed,scroll_percentage_fixed
0,22779,9775331,0,0
1,22779,9775402,0,0
2,22779,9771051,0,0
3,22779,9770028,0,0
4,22779,9775371,0,0


In [265]:
user_articles_all.tail()

,user_id,article_id_fixed,read_time_fixed,scroll_percentage_fixed
258313,2539047,9755010,5.0,100.0
258314,2539047,9755980,17.0,100.0
258315,2539047,9755980,10.0,100.0
258316,2539047,9755648,3.0,19.0
258317,2539047,9755980,2.0,100.0


In [266]:
groups = user_articles_all.groupby(['article_id_fixed', 'user_id']).size()

groups[groups > 100]

article_id_fixed  user_id
4543921           2110847    279
9432404           388844     217
9758326           765092     114
9762058           1377811    103
9762249           2201343    103
dtype: int64

In [267]:
user_articles_max_read_time = user_articles_all.groupby(['user_id', 'article_id_fixed'])['read_time_fixed'].max().reset_index()

print(user_articles_max_read_time)

        user_id  article_id_fixed read_time_fixed
0         11313           9251369           299.0
1         11313           9730301           323.0
2         11313           9733713             7.0
3         11313           9737535          1038.0
4         11313           9738292            32.0
...         ...               ...             ...
397798  2589391           9770798            18.0
397799  2589391           9778351             0.0
397800  2589391           9778375             0.0
397801  2589391           9778413             0.0
397802  2589391           9778422             0.0

[397803 rows x 3 columns]


In [268]:
matrix = user_articles_max_read_time.pivot(index='article_id_fixed', columns='user_id', values='read_time_fixed')

matrix.head()

user_id,11313,13538,15430,19181,19568,19892,19993,21271,21774,22779,...,2573441,2574644,2575952,2578206,2579603,2581162,2583035,2583477,2585449,2589391
article_id_fixed,,,,,,,,,,,,,,,,,,,,,
3044020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3073151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3223114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3971783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3983641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [269]:
matrix.tail()

user_id,11313,13538,15430,19181,19568,19892,19993,21271,21774,22779,...,2573441,2574644,2575952,2578206,2579603,2581162,2583035,2583477,2585449,2589391
article_id_fixed,,,,,,,,,,,,,,,,,,,,,
9780162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9780181,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
9780193,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
9780195,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9780271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [270]:
matrix.fillna(0, inplace=True)

C:\Users\daniel.georgiev\AppData\Local\Temp\ipykernel_71280\1873464318.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matrix.fillna(0, inplace=True)


In [271]:
print(matrix.iloc[5626, len(matrix.columns)-1])

0.0


In [272]:
matrix.tail()

user_id,11313,13538,15430,19181,19568,19892,19993,21271,21774,22779,...,2573441,2574644,2575952,2578206,2579603,2581162,2583035,2583477,2585449,2589391
article_id_fixed,,,,,,,,,,,,,,,,,,,,,
9780162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9780181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9780193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9780195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9780271,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [273]:
df1 = matrix.copy()
number_neighbors = 30
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(matrix.values)
distances, indices = knn.kneighbors(matrix.values, n_neighbors=number_neighbors)


In [274]:
def recommend_articles(user, articles_inview):
  recommended_articles = []

  for a in articles_inview:
    article_index = matrix.index.tolist().index(a)
    predicted_rating = df1.iloc[article_index, df1.columns.tolist().index(user)]
    
    recommended_articles.append((a, predicted_rating))

  sorted_ra = sorted(recommended_articles, key=lambda x:x[1], reverse=True)
  
  return sorted_ra[0][0]
#   print('The list of the Recommended articles \n')
#   rank = 1
#   for recommended_article in sorted_ra:
    
#     print('{}: {} - predicted rating:{}'.format(rank, recommended_article[0], recommended_article[1]))
#     rank = rank + 1

In [287]:
def article_recommender(user, num_neighbours, articles_inview):
    number_neighbors = num_neighbours

    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(matrix.values)
    distances, indices = knn.kneighbors(matrix.values, n_neighbors=number_neighbors)

    user_index = matrix.columns.tolist().index(user)

    for article_id in articles_inview:
        a = matrix.index.tolist().index(article_id)
        print(sum(matrix.loc[article_id]))

        if matrix.iloc[a, user_index] <= 0:
            sim_articles = indices[a].tolist()
            article_distances = distances[a].tolist()

            if a in sim_articles:
                id_article = sim_articles.index(a)
                sim_articles.remove(a)
                article_distances.pop(id_article)
            else:
                sim_articles = sim_articles[:num_neighbours-1]
                article_distances = article_distances[:num_neighbours-1]

            article_similarity = [1-x for x in article_distances]
            article_similarity_copy = article_similarity.copy()
            nominator = 0

            for s in range(0, len(article_similarity)):
                if matrix.iloc[sim_articles[s], user_index] <= 0:
                    if len(article_similarity_copy) == (number_neighbors-1):
                        article_similarity_copy.pop(s)
                    else:
                        article_similarity_copy.pop(s-(len(article_similarity)-len(article_similarity_copy)))
                else:
                    nominator = nominator + article_similarity[s]*matrix.iloc[sim_articles[s], user_index]

            if len(article_similarity_copy) > 0:
                if sum(article_similarity_copy) != 0:
                    predicted_r = nominator/sum(article_similarity_copy)
                else:
                    predicted_r = -1
            else:
                predicted_r = -1
                
            df1.iloc[a, user_index] = predicted_r

    recommend_articles(user, articles_inview)

In [288]:
article_recommender(1001055, 3000, [9775202, 9776855, 9776688, 9771995, 9776583, 9776553])

0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 